<a href="https://colab.research.google.com/github/onlyabhilash/spark-nlp-transformers/blob/main/HuggingFace_in_Spark_NLP_DeBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20DeBERTa.ipynb)

## Import DeBERTa models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only available in `Spark NLP 3.4.2` and above. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for DeBERTa from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use DeBERTa models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.8.0` version and Transformers on `4.18.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- DebertaV2Tokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers==4.17.0 tensorflow==2.8.0 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [microsoft/deberta-v3-xsmall](https://huggingface.co/microsoft/deberta-v3-xsmall) model from HuggingFace as an example
- In addition to `TFDebertaV2Model` we also need to save the `DebertaV2Tokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- Since `microsoft/deberta-v3-xsmall` model is PyTorch we will use `from_pt=True` param to convert it to TensorFlow

In [ ]:
from transformers import DebertaV2Tokenizer, TFDebertaV2Model
import tensorflow as tf

MODEL_NAME = 'microsoft/deberta-v3-xsmall'

DebertaV2Tokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFDebertaV2Model.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFDebertaV2Model.from_pretrained(MODEL_NAME, from_pt=True)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

In [ ]:
!ls -l {MODEL_NAME}_tokenizer

- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for

In [ ]:
# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spm.model {MODEL_NAME}/saved_model/1/assets

## Import and Save DeBERTa in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DeBertaEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` only accepts local paths and not distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. That is why we use `write.save` so we can use `.load()` from any file systems.


In [ ]:
from sparknlp.annotator import *

deberta = DeBertaEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('deberta_v3_xsmall') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
deberta.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DeBERTa model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊 

In [ ]:
deberta_loaded = DeBertaEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [ ]:
deberta_loaded.getStorageRef()

That's it! You can now go wild and use hundreds of DeBERTa models from HuggingFace 🤗 in Spark NLP 🚀 
